<a href="https://colab.research.google.com/github/sswastik630/ML_Algorithms/blob/main/02_insurance_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('02-insurance-linear-regression')
jovian.set_colab_id('10XQRJcggTtv-cyYTMuglJ2QakG8XWPfo')

     |████████████████████████████████| 71kB 4.9MB/s 


# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [2]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [4]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [3]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [6]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [7]:
your_name = "Swastik" # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [8]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [10]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
665,43,male,45.29140,2,yes,41283.617488
579,25,female,27.92335,0,no,3110.296609
84,37,female,41.41200,2,yes,38641.423430
244,63,female,33.01060,0,yes,28637.470632
1307,32,male,33.46280,4,yes,20828.304436


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [11]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [13]:
num_cols = dataframe.shape[1]
print(num_cols)

6


**Q: What are the column titles of the input variables?**

In [14]:

input_cols=[x for x in dataframe.columns if x != 'charges']
input_cols

['age', 'sex', 'bmi', 'children', 'smoker']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [24]:
# print(dataframe.dtypes)
categorical_cols=[x for x in dataframe.select_dtypes(include='object')]
categorical_cols

['sex', 'smoker']

**Q: What are the column titles of output/target variable(s)?**

In [25]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [26]:
dataframe['charges'].describe()

count     1271.000000
mean     12983.489566
std      11845.412394
min       1088.217683
25%       4604.297496
50%       9128.674850
75%      16335.447931
max      61857.315170
Name: charges, dtype: float64

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [27]:
!pip install jovian --upgrade -q

In [28]:
import jovian

In [29]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
API KEY: ··········
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/sswastik630/02-insurance-linear-regression


'https://jovian.ai/sswastik630/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [35]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [36]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
print(inputs_array) 
print(targets_array)

[[43.       1.      45.2914   2.       1.     ]
 [25.       0.      27.92335  0.       0.     ]
 [37.       0.      41.412    2.       1.     ]
 ...
 [47.       1.      30.2379   1.       1.     ]
 [35.       1.      28.7147   1.       0.     ]
 [43.       0.      30.0713   1.       1.     ]]
[[41283.617488 ]
 [ 3110.2966095]
 [38641.42343  ]
 ...
 [21319.316593 ]
 [ 4971.459229 ]
 [21118.202031 ]]


**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [37]:
inputs = torch.tensor(inputs_array,dtype=torch.float32)
targets =torch.tensor(targets_array, dtype=torch.float32)

In [38]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [39]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [40]:
val_percent = 0.2 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset,(train_size,val_size))# Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [41]:
batch_size = 128

In [42]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [43]:

for xb, yb in train_loader:
    print("input shape", xb.shape)
    print("target shape", yb.shape)
    print("inputs:", xb)

    print("targets:", yb)
    break

input shape torch.Size([128, 5])
target shape torch.Size([128, 1])
inputs: tensor([[23.0000,  1.0000, 37.7587,  3.0000,  1.0000],
        [35.0000,  0.0000, 33.3498,  0.0000,  1.0000],
        [60.0000,  0.0000, 42.8460,  0.0000,  0.0000],
        [35.0000,  1.0000, 33.0106,  2.0000,  1.0000],
        [55.0000,  1.0000, 39.2700,  0.0000,  0.0000],
        [35.0000,  1.0000, 21.2534,  1.0000,  0.0000],
        [27.0000,  1.0000, 33.9150,  0.0000,  1.0000],
        [26.0000,  0.0000, 35.6048,  2.0000,  0.0000],
        [38.0000,  1.0000, 41.2930,  2.0000,  0.0000],
        [19.0000,  1.0000, 40.5790,  0.0000,  0.0000],
        [45.0000,  0.0000, 29.9583,  2.0000,  0.0000],
        [56.0000,  0.0000, 49.8729,  0.0000,  0.0000],
        [53.0000,  0.0000, 43.8634,  3.0000,  1.0000],
        [18.0000,  1.0000, 27.3581,  0.0000,  0.0000],
        [36.0000,  0.0000, 26.3407,  3.0000,  0.0000],
        [50.0000,  1.0000, 43.0780,  0.0000,  0.0000],
        [31.0000,  0.0000, 31.6778,  0.0000, 

Let's save our work by committing to Jovian.

In [44]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/sswastik630/02-insurance-linear-regression


'https://jovian.ai/sswastik630/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [45]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [46]:

class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size,output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                         # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out,targets)                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out,targets)                     # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 1000 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [47]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [48]:

list(model.parameters())

[Parameter containing:
 tensor([[-0.0859, -0.2821,  0.1091, -0.0399,  0.1177]], requires_grad=True),
 Parameter containing:
 tensor([-0.3507], requires_grad=True)]

One final commit before we train the model.

In [49]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/sswastik630/02-insurance-linear-regression


'https://jovian.ai/sswastik630/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [50]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [51]:
result = evaluate(model,val_loader) # Use the the evaluate function
print(result)

{'val_loss': 12768.6484375}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [52]:
epochs = 10000
lr = 1e-4
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1000], val_loss: 10526.7041
Epoch [2000], val_loss: 8967.6143
Epoch [3000], val_loss: 8045.8750
Epoch [4000], val_loss: 7543.2998
Epoch [5000], val_loss: 7312.2988
Epoch [6000], val_loss: 7221.4858
Epoch [7000], val_loss: 7183.2969
Epoch [8000], val_loss: 7159.6133
Epoch [9000], val_loss: 7143.8906
Epoch [10000], val_loss: 7131.8008


In [53]:
epochs = 10000
lr = 1e-3
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1000], val_loss: 7119.1299
Epoch [2000], val_loss: 7106.4775
Epoch [3000], val_loss: 7094.0879
Epoch [4000], val_loss: 7081.7939
Epoch [5000], val_loss: 7069.2080
Epoch [6000], val_loss: 7056.6074
Epoch [7000], val_loss: 7043.9121
Epoch [8000], val_loss: 7031.2188
Epoch [9000], val_loss: 7018.5537
Epoch [10000], val_loss: 7006.1562


In [54]:
epochs = 5000
lr = 1e-5
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1000], val_loss: 6993.9268
Epoch [2000], val_loss: 6981.5811
Epoch [3000], val_loss: 6969.0601
Epoch [4000], val_loss: 6956.8237
Epoch [5000], val_loss: 6945.0063
Epoch [6000], val_loss: 6933.6973
Epoch [7000], val_loss: 6922.8057
Epoch [8000], val_loss: 6912.1426
Epoch [9000], val_loss: 6901.4541
Epoch [10000], val_loss: 6890.8828


In [55]:
epochs = 10000
lr = 1e-4
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1000], val_loss: 6880.5928
Epoch [2000], val_loss: 6870.5518
Epoch [3000], val_loss: 6860.8066
Epoch [4000], val_loss: 6851.2080
Epoch [5000], val_loss: 6841.6279
Epoch [6000], val_loss: 6832.0508
Epoch [7000], val_loss: 6822.5342
Epoch [8000], val_loss: 6813.0000
Epoch [9000], val_loss: 6803.5459
Epoch [10000], val_loss: 6794.2583


In [56]:
epochs = 10000
lr = 1e-4
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1000], val_loss: 6785.3022
Epoch [2000], val_loss: 6775.9385
Epoch [3000], val_loss: 6766.7949
Epoch [4000], val_loss: 6758.1729
Epoch [5000], val_loss: 6749.6729
Epoch [6000], val_loss: 6742.1816
Epoch [7000], val_loss: 6734.9531
Epoch [8000], val_loss: 6727.7446
Epoch [9000], val_loss: 6721.0186
Epoch [10000], val_loss: 6714.4082


**Q: What is the final validation loss of your model?**

In [57]:
val_loss = history5[-1]['val_loss']

Let's log the final validation loss to Jovian and commit the notebook

In [58]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [59]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/sswastik630/02-insurance-linear-regression


'https://jovian.ai/sswastik630/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [62]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [63]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([55.0000,  0.0000, 41.8880,  0.0000,  1.0000])
Target: tensor([43091.0898])
Prediction: tensor([10738.7334])


In [64]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([24.0000,  0.0000, 24.4188,  0.0000,  1.0000])
Target: tensor([14134.7344])
Prediction: tensor([4593.7144])


In [65]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([28.0000,  1.0000, 28.5362,  3.0000,  1.0000])
Target: tensor([17133.2500])
Prediction: tensor([5368.5732])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [66]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/sswastik630/02-insurance-linear-regression
[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/sswastik630/02-insurance-linear-regression


'https://jovian.ai/sswastik630/02-insurance-linear-regression'